# Obteniendo el **número de dosis aplicada** contra COVID-19 por semana epidemiológica de todo el Perú

## 0. Cargar librerías

In [45]:
import pandas as pd
import numpy as np

import functions as fn

In [51]:
def vac_dose(vac_url):
    """
    Función que toma la dirección del dataset de vacunados y devuelve el TOTAL DE DOSIS APLICADAS 
    (1ra,2nda y 3era dosis) por semana epidemiológica
    """
    vac_cols = ['fecha_vacunacion', 'dosis']                     
    df_vac = fn.read_largeCSV_file(vac_url, ',', vac_cols)    
    lst_vac = fn.df_into_chunks(df_vac)               
 
    for idx, chunk in enumerate(lst_vac):
        chunk = fn.variable_fecha_ymd(chunk, 'fecha_vacunacion')

    for idx, chunk in enumerate(lst_vac):
        chunk = fn.date_to_epiweek(chunk, "fecha_vacunacion")
        
    return lst_vac 
    

## 1. Cargar el dataset de **personas vacunadas (1era y 2nda dosis)** por semana epidemiológica

In [52]:
vac_url = "RawData/TB_VACUNACION_COVID19.csv"
vacDose = vac_dose(vac_url)
del vac_url

/home/xxotto/Documents/GitHub/covid19-peru/functions.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,date_name] = pd.to_datetime(df.loc[:,date_name], format = '%d/%m/%Y')


TypeError: 'NoneType' object is not subscriptable

## 0. Usamos la data incial de vacunados por departamento (vacDose)

In [50]:
print(vacDose[0].head())

   dosis  epi_year  epi_week
0      2      2021        29
1      2      2021        24
2      2      2021        23
3      2      2021        30
4      1      2021        18


In [ ]:
def vacDose_chunks(dfs_vac):
    """
    Devuelve un dataframe con el TOTAL DE VACUNADOS por DEPARTAMENTO 
    del PERÚ (recibe una lista de dataframes o chunks)
    """
    var_holder = {}     # Diccionario para guardar nombres                                             
    lst_epi_vac = []    # Lista de dfs para cada sumatoria de chunks
                                         
    for i, chunk in enumerate(dfs_vac):
        var_holder['epi_vac_' + str(i)]= pd.crosstab(index = [chunk['epi_year'], chunk['epi_week']],
                                                     columns = [chunk['dosis']])
        lst_epi_vac.append(var_holder['epi_vac_' + str(i)])

    merged_epivac = pd.concat(lst_epi_vac, axis=1)  # Merge all dfs
    merged_epivac = merged_epivac.fillna(0).astype(np.int64)
    merged_epivac = merged_epivac.groupby(level=0, axis=1).sum()

    return merged_epivac

In [ ]:
vacDosexsemEpi = vacDose_chunks(vacDose)
print(vacDosexsemEpi.head(50))

Guardar el dataframe:

In [ ]:
vacDosexsemEpi.to_csv('Data/dosis_vacuna_x_semanaEpi.csv')